## Предобработка: лемматизация и разметка ЧР по universal dependencies 

In [ ]:
# используем udpipe (vs pymystem)

In [2]:
!pip install ufal.udpipe

In [3]:
from ufal.udpipe import Model, Pipeline

In [4]:
import codecs

In [4]:
!pip install wget

In [5]:
modelfile = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\udpipe_syntagrus.model"

#### Скрипт для предобработки 

In [ ]:
# https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py
'''
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_NOUN трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).
Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
'''

In [6]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline

In [7]:
# вспомогательные функции для очистки текста

def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u2241\u224B\u2E2F\u0483', '\u223D', text)

    text = list_replace('\u00C4', 'A', text)  # латинская
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text


In [8]:
# функция для запуска предобработки 

def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [9]:
# собственно обработка 

standard_library.install_aliases()

def tag_ud(text, outputfile, process_pipeline):
    # текст нужно передать в виде строки с переносами строк \n
    outputlist = []

    print('Processing input...', file=sys.stderr)
    for line in text.split("\n"): 
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        
        # записываем строку в лист
        outputlist.append(' '.join(output))
    
    with codecs.open(outputfile, "w", "utf-8") as output:
        output.write("\n".join(outputlist))

In [37]:
# запуск (сразу на всех файлах)
modelfile='C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\udpipe_syntagrus.model'
print('\nLoading the model...', file=sys.stderr)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

common_path = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\"
"""
instrumenty_input_path = "!instrumenty\\raw_texts\\without_punctuation\\"
instrumenty_output_path = "!instrumenty\\preprocessed\\"
keys1 = ["britva", "karandash", "kosa", "lom", "lopata", "metla", "nozhnitsy", "schotka", "topor", "venik", "veslo"]
for key in keys1:
    print(key)
    instrument_input_files = os.listdir(common_path + instrumenty_input_path + key + "\\")
    for file in instrument_input_files:
        if file.endswith(".txt") and file != "bad.txt":
            outputfile = common_path + instrumenty_output_path + key+ "\\" + file
            with codecs.open(common_path + instrumenty_input_path + key + "\\" + file, 'r', encoding='utf-8') as input_f:
                text = input_f.read()
                tag_ud(text, outputfile, process_pipeline)
                print(file)
"""

raznoje_input_path = "!raznoje\\raw_texts\\without_punctuation\\"
raznoje_output_path = "!raznoje\\preprocessed\\"
keys2 = ["dom", "glava", "luk", "organ", "vid"]
for key in keys2:
    print(key)
    raznoje_input_files = os.listdir(common_path + raznoje_input_path + key + "\\good\\")
    for file in raznoje_input_files:
        if file.endswith(".txt"):
            outputfile = common_path + raznoje_output_path + key + "\\" + file
            with codecs.open(common_path + raznoje_input_path + key + "\\good\\" + file, 'r', encoding='utf-8') as input_f:
                text = input_f.read() 
                # в "разном" убираем подчёркивания целевого слова: так не очень хорошо лемматизируется 
                text = text.replace("_", "")
                tag_ud(text, outputfile, process_pipeline)
                print(file)


Loading the model...
Processing input...


dom
{'r-concr_t-constr_top-contain',_'r-concr_t-org'}.txt


Processing input...


{'r-concr_t-constr_top-contain',_'r-concr_t-space'}.txt


Processing input...


...

{'r-abstr_t-ment'}.txt


Processing input...


{'r-abstr_t-perc_der-v'}.txt
{'r-concr_t-doc'}.txt


Processing input...
Processing input...


{'r-concr_t-workart'}.txt


#### Удаляем функциональные слова  

In [167]:
# из файлов убираем ADP SCONJ CCONJ PART INTJ SYM X AUX, 
# а также в разном - первые строки, начинающиеся с # (где есть) и номера строк в начале файлов
path_to_instrumenty = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!instrumenty\\preprocessed\\"
path_to_raznoje = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\preprocessed\\"

keys1 = ["britva", "karandash", "kosa", "lom", "lopata", "metla", "nozhnitsy", "schotka", "topor", "venik", "veslo"]
keys2 = ["dom", "glava", "luk", "organ", "vid"]


for key in keys1:
    files = os.listdir(path_to_instrumenty + key + "\\")
    for file in files:
        with codecs.open(path_to_instrumenty + key + "\\" + file, "r", "utf-8") as input_f:
            with codecs.open(path_to_instrumenty + "without_some_POS\\" + key + "\\" + file, "w", "utf-8") as output_f: 
                lines = input_f.read().split("\n")
                new_lines = []
                for line in lines:
                    new_line = []
                    for word in line.split():
                        if (word.endswith("_PART") or word.endswith("_SCONJ") or word.endswith("_CCONJ") or \
                            word.endswith("_X") or word.endswith("_ADP") or word.endswith("_INTJ") or \
                            word.endswith("_SYM") or word.endswith("_AUX")):
                            pass
                        else:
                            new_line.append(word)
                    new_lines.append(" ".join(new_line))
                output_f.write("\n".join(new_lines))        
"""
for key in keys2:
    files = os.listdir(path_to_raznoje + key + "\\")
    for file in files:
        with codecs.open(path_to_raznoje + key + "\\" + file, "r", "utf-8") as input_f:
            with codecs.open(path_to_raznoje + "without_some_POS\\" + key + "\\" + file, "w", "utf-8") as output_f: 
                lines = input_f.read().split("\n")
                new_lines = []
                for line in lines:
                    if line == "" or line.startswith("#"):  # на самом деле это условие уже не нужно
                        new_line = []
                        pass
                    else:
                        try:
                            line = line[line.index("_NUM")+5:]  # обрезаем номера строк
                        except:
                            print(key, file)
                            print(repr(line))
                        new_line = []
                        for word in line.split():
                            if (word.endswith("_PART") or word.endswith("_SCONJ") or word.endswith("_CCONJ") or \
                                word.endswith("_X") or word.endswith("_ADP") or word.endswith("_INTJ") or \
                                word.endswith("_SYM") or word.endswith("_AUX")):
                                pass
                            else:
                                new_line.append(word)
                    new_lines.append(" ".join(new_line))
                output_f.write("\n".join(new_lines))  
"""                            

'\n\nfor key in keys2:\n    files = os.listdir(path_to_raznoje + key + "\\")\n    for file in files:\n        with codecs.open(path_to_raznoje + key + "\\" + file, "r", "utf-8") as input_f:\n            with codecs.open(path_to_raznoje + "without_some_POS\\" + key + "\\" + file, "w", "utf-8") as output_f: \n                lines = input_f.read().split("\n")\n                new_lines = []\n                for line in lines:\n                    if line == "" or line.startswith("#"):  # на самом деле это условие уже не нужно\n                        new_line = []\n                        pass\n                    else:\n                        try:\n                            line = line[line.index("_NUM")+5:]  # обрезаем номера строк\n                        except:\n                            print(key, file)\n                            print(repr(line))\n                        new_line = []\n                        for word in line.split():\n                            if (word.e

In [139]:
# проверяем, что в каждой строке есть целевое слово - теперь да!
# для разного
translit_raznoje = {"dom":"дом",
"glava":"глава",
"luk":"лук",
"organ":"орган",
"vid":"вид"}

path_to_raznoje = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\preprocessed\\without_some_POS\\" 

for element in keys2:
    if element.startswith("NEW"):
        print("______________________" + element + "______________________")
        files = os.listdir(path_to_raznoje + element + "\\")
        word_we_look_for = translit_raznoje[element]
        for file in files:
            print(file)
            j = 0
            with codecs.open(path_to_raznoje + element + "\\" + file, "r", "utf-8") as file_to_prove:
                lines = file_to_prove.read().split("\n")
                for i, line in enumerate(lines):
                    if j < 10:
                        if ((word_we_look_for+"_NOUN") not in line.split()) and (word_we_look_for+"_PROPN" not in line.split()):
                            print(i, line)
                            j += 1
            print()
        print()

### Исправление ошибок 

#### Для разного: сначала составляем словарь

In [58]:
path = 'C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\'
os.chdir(path)
files = os.listdir()

In [59]:
dict_with_files = dict()
for file in files:
    if file.endswith(".txt") and file != "DICT.txt":
        print(file)
        with codecs.open(file, "r", "utf-8") as opened:
            file = file[:-4]
            dict_with_files[file] = []
            lines = opened.readlines()
            for i, line in enumerate(lines):
                list_for_line = []
                annotation_index = 0
                while True:
                    try:
                        annotation_index = line.index("ana lex", annotation_index+6)  # начало аннотации в целом
                        gr_index = line.index("gr=", annotation_index)
                        word_index = line.index("/>", annotation_index)
                        try:
                            SEMF_index = line.index("SEMF=", annotation_index)
                        except ValueError:
                            try: 
                                SEMF_index = line.index("SEMF2=", annotation_index)
                            except ValueError:
                                SEMF_index = 0
                        try:
                            sem_index = line.index("sem=", annotation_index)
                        except:
                            sem_index = 0
                    except:
                        break

                    lexema = line[annotation_index+9:line.index("'",annotation_index+9)].strip()
                    grammar = line[gr_index+4:line.index("'",gr_index+4)].strip()
                    try:
                        word_end = line.index("<", word_index)
                    except ValueError:
                        word_end = len(line)-1
                        print(i, line)
                        
                    word = line[word_index+2:word_end].replace("`", "").strip()

                    if (word_index > sem_index) and (sem_index != 0):
                        sem = line[sem_index+5:line.index("/>", sem_index)-1].strip()
                    elif (word_index > SEMF_index) and (SEMF_index != 0):
                        sem = line[SEMF_index+6:line.index("/>", SEMF_index)-1].strip()
                    else:
                        sem = ""
                    
                    list_for_line.append([lexema, grammar, sem, word])
                dict_with_files[file].append(list_for_line)      

dom.txt
glava.txt
luk.txt
organ.txt
vid.txt


In [60]:
# создаём класс для неоднозначных слов
class Word_to_Disambiguate(object):
    
    def __init__(self, lemmas, sem_set, string, num_of_string, num_in_string):
        self.lemmas = lemmas  # set с леммами (set - потому что их мб несколько, из разных аннотаций)
        self.sem_set = sem_set  # set с семантикой
        self.string = string  # строка из raw файла
        self.num_of_string = num_of_string  # номер строки в файле (в raw и в original равны)
        self.num_in_string = num_in_string  # номер слова в строке, в raw и original равны, если считать те, где word[3] != ''
    
    def __repr__(self):
        string_to_return = "lemmas: %s \n sems: %s \n num_in_string: %s \n %s - %s" % (str(self.lemmas), 
                                                                                       str(self.sem_set),
                                                                                      str(self.num_in_string), 
                                                                                      str(self.num_of_string), 
                                                                                          self.string)
        return(string_to_return)

In [61]:
# ДЛЯ РАЗНЫХ
translit = {"dom":"Дом",
"glava":"Глава",
"luk":"Лук",
"organ":"Орган",
"vid":"Вид"}

In [62]:
# словарь, в котором будут все целевые слова
dict_with_words_to_disambiguate = dict()

for name in dict_with_files:
    dict_with_words_to_disambiguate[name] = []
    with codecs.open("C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\raw_texts\\without_punctuation\\" + 
                    name + ".txt", "r", "utf-8") as raw_text:
        raw_lines = raw_text.readlines()
        print(name)
        for j, line in enumerate(dict_with_files[name]):
            sem_set_for_one_word = set()
            lemmas = set()
            num_of_string = j
            string = raw_lines[j]
            num_in_string = -1  # сначала будет равен 0 (тк +1), а потом будем искать следующее вхождение слова, начиная с него
            i = 0
            while i<len(line):  # проходимся по словам
                # сначала разбираемся с инстансами, у которых было несколько аннотаций
                if line[i][3] == "" and (line[i][0] == translit[name] or line[i][0] == translit[name].lower()):
                    lemmas.add(line[i][0])
                    for sem in line[i][2].split("|"):
                        sem_set_for_one_word.add(sem.replace("'","").replace("SEMF=","").strip())  
                    for counter in range(1, 10):
                        if line[i+counter][3] == "":
                            lemmas.add(line[i+counter][0])
                            for sem in line[i][2].split("|"):
                                sem_set_for_one_word.add(sem.replace("'","").replace("SEMF=","").strip())
                        else:
                            lemmas.add(line[i+counter][0])
                            for sem in line[i][2].split("|"):
                                sem_set_for_one_word.add(sem.replace("'","").replace("SEMF=","").strip())
                            # здесь предусматриваем случай, когда в строке мб несколько одинаковых словоформ: поэтому 
                            # в каждой новой строке начинаем искать с нуля, а потом - с предыдущего вхождения
                            try:
                                num_in_string = string.split().index(line[i+counter][3], num_in_string+1)
                            except ValueError:  # не нашли в строке слово
                                print("Multiple annotations")
                                print(num_of_string, line[i+counter][3], string)
                            
                            # закончили с этим словом
                            dict_with_words_to_disambiguate[name].append(Word_to_Disambiguate(lemmas, sem_set_for_one_word, 
                                                                                              string, num_of_string, 
                                                                                              num_in_string))
                            sem_set_for_one_word = set()
                            lemmas = set()
                            i += counter + 1
                            break
                            
                elif (line[i][0] == translit[name] or line[i][0] == translit[name].lower()): # если у слова одна аннотация
                    lemmas = set([line[i][0]])
                    for sem in line[i][2].split("|"):
                        sem_set_for_one_word.add(sem.replace("'","").replace("SEMF=","").replace("SEMF2=","").strip())
                    try:
                        num_in_string = string.split().index(line[i][3], num_in_string+1)
                    except ValueError:  # не нашли в строке слово
                        print("One annotation")
                        print(str(num_of_string) + " /"+line[i][3]+"/ " + string)
                    dict_with_words_to_disambiguate[name].append(Word_to_Disambiguate(lemmas, sem_set_for_one_word, string,
                                                                                     num_of_string, num_in_string))
                    # закончили с этим словом
                    sem_set_for_one_word = set()
                    lemmas = set()
                    i += 1
                    
                else:  # это не целевое слово
                    i += 1

dom
glava
luk
organ
vid


In [63]:
# создаём новый словарь, чтобы там были только семы без примеров
words_to_disambiguate_wo_exs = dict()
for name in dict_with_words_to_disambiguate:
    words_to_disambiguate_wo_exs[name] = []
    for instance in dict_with_words_to_disambiguate[name]:
        new_sem_set = set()
        for element in instance.sem_set:
            try:
                ex_ind = element.index("Ex")
            except ValueError:
                ex_ind = len(element)
            new_sem = element[:ex_ind].strip()
            new_sem_set.add(new_sem)
        new_instance = Word_to_Disambiguate(instance.lemmas, new_sem_set, instance.string, 
                                            instance.num_of_string, instance.num_in_string)
        words_to_disambiguate_wo_exs[name].append(new_instance)  

In [67]:
for name in words_to_disambiguate_wo_exs:
    for i, instance in enumerate(words_to_disambiguate_wo_exs[name]):
        if i < 5:
            print(instance)

lemmas: {'дом'} 
 sems: {'r:concr t:constr top:contain'} 
 num_in_string: 2 
 0 - Добротный деревянный дом рядом с крыльцом берёза а от дома дорога ведёт к Богоявленской церкви

lemmas: {'дом'} 
 sems: {'r:concr t:constr top:contain'} 
 num_in_string: 9 
 0 - Добротный деревянный дом рядом с крыльцом берёза а от дома дорога ведёт к Богоявленской церкви

lemmas: {'дом'} 
 sems: {'r:concr t:constr top:contain'} 
 num_in_string: 3 
 1 - Это и есть дом где Андрей Тарковский провёл довоенное и военное детство

lemmas: {'дом'} 
 sems: {'r:concr t:space', 'r:concr t:constr top:contain'} 
 num_in_string: 2 
 2 - Входим в дом открыв дверь попадаем в кухню печь налево окно рядом небольшой стол керосиновая лампа и три деревянных стула

lemmas: {'дом'} 
 sems: {'r:concr t:org', 'r:concr t:constr top:contain'} 
 num_in_string: 1 
 3 - Посещение дома музея лишь часть обширной программы Дней Тарковского в Юрьевце

lemmas: {'глава'} 
 sems: {'r:concr pt:partb pc:hum'} 
 num_in_string: 53 
 0 - Крутизн

In [79]:
# функция для преобразования вложенных списков в невложенные
from collections import Iterable

def flatten_list(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x

In [82]:
with codecs.open(path + name + "\\" + file_with_name_as_set, "r", "utf-8") as input_f:
    input_lines = input_f.read().split("\n")
    print(type(input_lines))
    print(input_lines[0])

<class 'list'>
x_NUM посещение_NOUN дома_NOUN музей_NOUN лишь_PART часть_NOUN обширный_ADJ программа_NOUN день_PROPN  тарковский_PROPN в_ADP юрьевец_PROPN 


In [136]:
path = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\preprocessed\\"

for i, name in enumerate(words_to_disambiguate_wo_exs):  # dom 
    sem_sets_as_file_names = os.listdir(path + name + "\\")
    for file_with_name_as_set in sem_sets_as_file_names:
        set_as_string = file_with_name_as_set[:-4].replace("_", " ").replace("-",":")
        set_as_set = eval(set_as_string)  # convert a string representation into an actual set
        
        # создаём old_raw_files, если мы вносили изменения и, следовательно, есть старый файл (начинающийся на _)
        old_one = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\raw_texts\\without_punctuation\\" + \
        name + "\\good\\_" + file_with_name_as_set
        there_is_old_one = False
        if os.path.isfile(old_one):
            there_is_old_one = True
            with codecs.open(old_one, "r", "utf-8") as old_raw_file:
                old_raw_lines = old_raw_file.read().split("\n")
                for q, line in enumerate(old_raw_lines):
                    old_raw_lines[q] = line.replace("_", "")
        
        
        # open raw file - to find index of strings where to replace target words
        with codecs.open("C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\raw_texts\\without_punctuation\\" + 
                        name + "\\good\\" + file_with_name_as_set, "r", "utf-8") as raw_file:
            raw_lines = raw_file.read().split("\n")
            for l, line in enumerate(raw_lines):
                # убираем подчёркивания слов, чтобы мы потом могли найти строку "instance.num_of_string + instance.string" в
                # этом списке и запомнить её индекс
                raw_lines[l] = line.replace("_", "")
            
        with codecs.open(path + name + "\\" + file_with_name_as_set, "r", "utf-8") as pos_input_f:
            input_lines = pos_input_f.read().split("\n")

        for j, line in enumerate(input_lines):
            input_lines[j] = line.split()  # теперь элементы списка - списки со словами в соответствующей строке
        # но ещё нужно разделить по ::, потому что так склеиваются имена собственные
        # тогда, правда, мы потеряем информацию о именах собственных, поэтому нужно будет это отдельно потом обработать
        for k, line in enumerate(input_lines):
            for l, word in enumerate(input_lines[k]):
                input_lines[k][l] = word.split("::")
            # приводим разбиения к одному уровню, без вложенных списков, чтобы индексы соответствовали
            input_lines[k] = list(flatten_list(input_lines[k])) 

        for s, line in enumerate(input_lines):
            if len(input_lines[s]) != len(raw_lines[s].split()):
                print(name)
                print(file_with_name_as_set)
                print(raw_lines[s].split()[:10])
                print(input_lines[s][:10])
                print()
                raise ValueError("lengths are not the same")

        with codecs.open(path + name + "\\NEW_" + file_with_name_as_set, "w", "utf-8") as output_f:
            for p, instance in enumerate(words_to_disambiguate_wo_exs[name]):
                if (instance.sem_set == set_as_set):
                    try:
                        num_of_string = raw_lines.index(str(instance.num_of_string) + " " + instance.string[:-1])
                    except ValueError:
                        try:
                            num_of_string = raw_lines.index("-" + str(instance.num_of_string) + 
                                                            " " + instance.string[:-1])
                        except ValueError: 
                            # Строки instance.string нет raw_lines, потому что мы в неё вставляли дополнительные пробелы и
                            # там она уже изменённая. Чтобы получить индекс instance.string в raw lines, мы
                            # должны найти её индекс в ещё не изменённом файле: old_raw_lines
                                try:
                                    num_of_string = old_raw_lines.index(str(instance.num_of_string) + 
                                                                        " " + instance.string[:-1])
                                except ValueError:
                                    try:
                                        num_of_string = old_raw_lines.index("-" + str(instance.num_of_string) + 
                                                                            " " + instance.string[:-1])
                                    except:
                                        print(name, file_with_name_as_set)
                                        print("! В OLD_RAW_LINES ошибка")
                                        raise ValueError(str(instance.num_of_string) + " " + instance.string[:-1])

                    try:
                        if there_is_old_one:  # если мы в файл добавляли пробелы
                            # по старому файлу, по индексу слова находим форму, в которой оно стоит (сейчас индекс мб больше) 
                            word_form_we_are_searching = old_raw_lines[num_of_string].split()[instance.num_in_string+1]
                            # теперь ищем эту форму (её индекс) в новом файле в той же строке, начиная с 
                            # её индекса в первоначальной строке (а не сначала)
                            try:
                                its_index = raw_lines[num_of_string].split().index(word_form_we_are_searching, 
                                                                                   instance.num_in_string + 1)
                            except ValueError: # not in the list
                                print(name, file_with_name_as_set)
                                print(raw_lines[num_of_string])
                                print(word_form_we_are_searching)
                                
                            # когда нашли, теперь можем заменять в input lines, потому что там индекс такой же
                            input_lines[num_of_string][its_index] = translit[name].lower() + "_NOUN"
                        else:
                            input_lines[num_of_string][instance.num_in_string+1] = translit[name].lower() + "_NOUN"
                    except IndexError:
                        print(name)
                        print(instance)
                        print(num_of_string)
                        print(input_lines[num_of_string])
                        raise IndexError("Smth is wrong here")

            for m, line in enumerate(input_lines):
                for n, word in enumerate(line):
                    if "_" not in word:
                        line[n] = word + "_PROPN"  # так как разделили имена собственные, у некоторых может не быть тега
                input_lines[m] = " ".join(line)
            output_f.write("\n".join(input_lines))

glava {'r-concr_t-hum'}.txt
759 Вместо администрации одного сельского округа будет 10 15глав администраций во входящих в него деревнях насколько же будут раздуты штаты управленческого аппарата При новом дроблении органов МСУ война между муниципалитетами вспыхнет с новой силой
10-15глав


#### Для инструментов 

In [140]:
translit_instruments = {"britva":"бритва",
"karandash":"карандаш", 
"kosa":"коса", 
"lom":"лом", 
"lopata":"лопата", 
"metla":"метла", 
"nozhnitsy":"ножницы", 
"schotka":"щетка", 
"topor":"топор", 
"venik":"веник", 
"veslo":"весло"}

In [147]:
path_to_instrumenty = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!instrumenty\\preprocessed\\"

keys1 = ["britva", "karandash", "kosa", "lom", "lopata", "metla", "nozhnitsy", "schotka", "topor", "venik", "veslo"]


In [165]:
# сначала просто смотрели на слова в предобработанных текстах, и если их лемма = транслиту файла, писали им тег _NOUN
# потом посмотрели на ошибки и пытаемся их вручную исправить

wrong_lemmas = {"britva":["бритво", "бритвый"],
"karandash":["карандаша", "карандаать"], 
"kosa":["косой", "кос"], 
"lom":["ломе", "лый"], 
"lopata":["лопат"], 
"metla":["метл", "метел", "метлый", "мётл", "метлоя"], 
"nozhnitsy":["ножница", "ножницам"], 
"schotka":["щеткий", "щетку", "щеток", "щеткой"], 
"topor":["----------"], 
"venik":["веника"], 
"veslo":["весл", "весел"]}

for key in keys1:
    files = os.listdir(path_to_instrumenty + key + "\\")
    for file in files:
        if file.startswith("NEW"):
            pass
        else:
            with codecs.open(path_to_instrumenty + key + "\\" + file, "r", "utf-8") as input_f:
                lines = input_f.read().split("\n")

            for l, line in enumerate(lines):
                splitted_line = line.split()
                
                for w, word in enumerate(splitted_line):
                    # сначала разделяем имена собственные, добавляя ко всем кроме последнего кусочка "_PROPN"
                    splitted_PROPN = word.split("::")
                    for n, propn in enumerate(splitted_PROPN):
                        if n != len(splitted_PROPN) - 1:
                            splitted_PROPN[n] = splitted_PROPN[n] + "_PROPN"
                    splitted_line[w] = splitted_PROPN
                    splitted_line = list(flatten_list(splitted_line)) # приводим вложенные листы на верхний уровень
                
                for w, word in enumerate(splitted_line):    
                    # теперь переходим к исправлению неправильных лемм и тегов
                    splitted_word = word.split("_")
                    if (splitted_word[0] == translit_instruments[key]) or (splitted_word[0] in wrong_lemmas[key]):
                        splitted_line[w] = translit_instruments[key] + "_NOUN"
                lines[l] = " ".join(splitted_line)

            with codecs.open(path_to_instrumenty + key + "\\NEWNEW_" + file, "w", "utf-8") as output_f:
                output_f.write("\n".join(lines))

In [166]:
# проверяем, что в каждой строке есть целевое слово
# для инструментов

for key in keys1:
    print(key)
    files = os.listdir(path_to_instrumenty + key + "\\")
    word_we_look_for = translit_instruments[key]
    for file in files:
        if file.startswith("NEWNEW_"):
            print(file)
            j = 0
            with codecs.open(path_to_instrumenty + key + "\\" + file, "r", "utf-8") as file_to_prove:
                lines = file_to_prove.read().split("\n")
                for i, line in enumerate(lines):
                    if j < 10:
                        if ((word_we_look_for+"_NOUN") not in line.split()):
                            print(i, line)
                            j += 1
        print()

britva
NEWNEW_r-concr_t-tool-instr_der-v.txt
569 


karandash
NEWNEW_r-concr_t-tool-instr_top-rod.txt
1852 


kosa
NEWNEW_r-concr_pt-additb_pc-hum_()_r-concr_t-tool-instr_()_r-concr_t-space.txt
1038 


lom
NEWNEW_r-concr_t-tool-instr_top-rod_der-v_()_r-concr_pt-aggr_sc-thing_der-v.txt
583 я_PRON взвешивать_VERB положение_NOUN подумать_VERB прикидывать_VERB и_CCONJ приказывать_VERB лий_PROPN в_ADP срочный_ADJ порядок_NOUN овладевать_VERB английский_ADJ разговорный_ADJ речь_NOUN
614 


lopata
NEWNEW_r-concr_t-tool-instr.txt
104 xx_NUM сентябрь_NOUN от_ADP калининградский_ADJ областной_ADJ дума_NOUN витаутас_PROPN представлять_VERB в_ADP первый_ADJ чтение_NOUN законопроект_NOUN о_ADP внесение_NOUN дополнение_NOUN и_CCONJ изменение_NOUN в_ADP федеральный_ADJ закон_NOUN о_ADP оружие_NOUN по_ADP вопрос_NOUN расширение_NOUN перечень_NOUN вид_NOUN гражданский_ADJ оружие_NOUN самооборона_NOUN самооборона_NOUN
111 витаутас_PROPN депутат_NOUN калининградский_ADJ областной_ADJ дума_NOUN дума_NOUN


### Здесь обрабатываем "плохие" (=неоднозначные) контексты - лемматизируем и присваиваем ЧР

In [2]:
path = "C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\!raznoje\\raw_texts\\without_punctuation\\"

In [10]:
modelfile='C:\\Users\\boss\\Documents\\Diploma\\RNC_Subcorpus\\udpipe_syntagrus.model'
print('\nLoading the model...', file=sys.stderr)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model...


In [14]:
keys = ["glava", "luk", "organ", "vid"]

In [19]:
# лемматизируем
output_path = "lemmatized_bad\\"
for key in keys:
    input_path = key + "\\bad\\"

    input_files = os.listdir(path + input_path)
    for file in input_files:
        if file.endswith(".txt"):
            outputfile = path + output_path + key + "_bad.txt"
            with codecs.open(path + input_path + file, 'r', encoding='utf-8') as input_f:
                text = input_f.read() 
                # в "разном" убираем подчёркивания целевого слова: так не очень хорошо лемматизируется 
                text = text.replace("_", "")
                tag_ud(text, outputfile, process_pipeline)
                print(file)

Processing input...


{'r-concr_t-text_pt-part_pc-text',_'r-concr_pt-partb_pc-hum'}.txt


Processing input...


{'r-concr_t-tool-weapon_top-arc',_'r-concr_t-food_pt-aggr_sc-part(plant)'}.txt
{'r-concr_t-org_hi-class',_'r-concr_pt-partb_pc-hum_pc-animal_hi-class'}.txt


Processing input...
Processing input...


{'r-abstr',_'r-abstr_der-v'}.txt


In [25]:
# исправляем ошибки лемматизации
translit = {"dom":"Дом",
"glava":"Глава",
"luk":"Лук",
"organ":"Орган",
"vid":"Вид"}

for key in keys:
    file_path = path + output_path + key + "_bad.txt"
    with codecs.open(file_path, "r", "utf-8") as file_with_errors:
        lines = file_with_errors.readlines()
    for i, line in enumerate(lines):
        splitted_line = line.split()
        for j, word in enumerate(splitted_line):
            if word.split("_")[0].lower() == translit[key].lower():
                splitted_line[j] = word.split("_")[0]+"_NOUN"
        lines[i] = " ".join(splitted_line)
    with codecs.open(path + output_path + "without_errors\\" + key + "_bad.txt", "a", "utf-8") as file_without_errors:
        file_without_errors.write("\n".join(lines))

In [26]:
# удаляем функциональные слова
for key in keys:
    files = os.listdir(path + output_path + "without_errors\\")
    for file in files:
        if file.endswith(".txt"):
            with codecs.open(path + output_path + "without_errors\\" + file, "r", "utf-8") as input_f:
                with codecs.open(path + output_path + "without_some_POS\\" + file, "w", "utf-8") as output_f: 
                    lines = input_f.read().split("\n")
                    new_lines = []
                    for line in lines:
                        try:
                            line = line[line.index("_NUM")+5:]  # обрезаем номера строк
                        except:
                            # строки, в которых нет номера
                            print(key, file)
                            print(repr(line))
                        new_line = []
                        for word in line.split():
                            if (word.endswith("_PART") or word.endswith("_SCONJ") or word.endswith("_CCONJ") or \
                                word.endswith("_X") or word.endswith("_ADP") or word.endswith("_INTJ") or \
                                word.endswith("_SYM") or word.endswith("_AUX")):
                                pass
                            else:
                                new_line.append(word)
                        new_lines.append(" ".join(new_line))
                    output_f.write("\n".join(new_lines))  

glava vid_bad.txt
'все_DET перемешать_VERB'
glava vid_bad.txt
''
luk vid_bad.txt
'все_DET перемешать_VERB'
luk vid_bad.txt
''
organ vid_bad.txt
'все_DET перемешать_VERB'
organ vid_bad.txt
''
vid vid_bad.txt
'все_DET перемешать_VERB'
vid vid_bad.txt
''
